In [ ]:
def normal_train(model,optimizer,task,total_epochs,summary_epochs):
  model.train()
  model.zero_grad()
  ceriation=nn.CrossEntropyLoss()
  losses=[]
  loss=0.
  for epoch in range(summary_epochs):
    imgs,labels=next(task.train_iter)
    imgs,labels=imgs.to(device),labels.to(device)
    outputs=model(imgs)
    ce_loss=ceriation(outputs,labels)

    optimizer.zero_grad()
    ce_loss.backward()
    optimizer.step()

    loss+=ce_loss.item()
    if (epoch+1)%50==0:
      loss=loss/50
      print('\r','train task {} [{}] loss:{:.3f} '.format(task.name,(total_epochs+epoch+1),loss),end=' ')
      losses.append(loss)
      loss=0.
  return model,optimizer,losses

In [ ]:
def ewc_train(model,optimizer,task,total_epochs,summary_epochs,ewc,lambda_ewc):
  model.train()
  model.zero_grad()
  ceriation=nn.CrossEntropyLoss()
  losses=[]
  loss=0.
  for epoch in range(summary_epochs):
    imgs,labels=next(task.train_iter)
    imgs,labels=imgs.to(device),labels.to(device)
    outputs=model(imgs)
    ce_loss=ceriation(outputs,labels)
    total_loss=ce_loss
    ewc_loss=ewc.penalty(model)
    total_loss+=lambda_ewc*ewc_loss

    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

    loss+=total_loss.item()
    if (epoch+1)%50==0:
      loss=loss/50
      print('\r','train task {} [{}] loss:{:.3f} '.format(task.name,(total_epochs+epoch+1),loss),end=' ')
      losses.append(loss)
      loss=0.

  return model,optimizer,losses

In [ ]:
def mas_train(model,optimizer,task,total_epochs,summary_epochs,mas_tasks,lambda_mas,alpha=0.8):
  model.train()
  model.zero_grad()
  ceriation=nn.CrossEntropyLoss()
  losses=[]
  loss=0.
  for epoch in range(summary_epochs):
    imgs,labels=next(task.train_iter)
    imgs,labels=imgs.to(device),labels.to(device)
    outputs=model(imgs)
    ce_loss=ceriation(outputs,labels)
    total_loss=ce_loss
    mas_tasks.reverse()
    if len(mas_tasks)>1:
      preprevious=1-alpha
      scalars=[alpha,preprevious]
      for mas,scalar in zip(mas_tasks[:2],scalars):
        mas_loss=mas.penalty(model)
        total_loss+=lambda_mas*mas_loss*scalar
    elif len(mas_tasks)==1:
      mas_loss=mas_tasks[0].penalty(model)
      total_loss+=lambda_mas*mas_loss
    else:
      pass
    
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

    loss+=total_loss.item()
    if (epoch+1)%50==0:
      loss=loss/50
      print('\r','train task {} [{}] loss:{:.3f} '.format(task.name,(total_epochs+epoch+1),loss),end=' ')
      losses.append(loss)
      loss=0.
  return model,optimizer,losses

In [ ]:
def scp_train(model,optimizer,task,total_epochs,summary_epochs,scp_tasks,lambda_scp,alpha=0.65):
  losses=[]
  loss=0.

  model.train()
  model.zero_grad()
  ceriation=nn.CrossEntropyLoss()

  for epoch in range(summary_epochs):
    imgs,labels=next(task.train_iter)
    imgs,labels=imgs.to(device),labels.to(device)
    outputs=model(imgs)
    ce_loss=ceriation(outputs,labels)
    total_loss=ce_loss
    scp_tasks.reverse()
    if len(scp_tasks)>1:
      preprevious=1-alpha
      scalars=[alpha,preprevious]
      for scp,scalar in zip(scp_tasks[:2],scalars):
        scp_loss=scp.penalty(model)
        total_loss+=lambda_scp*scp_loss*scalar
    elif len(scp_tasks)==1:
      scp_loss=scp_tasks[0].penalty(model)
      total_loss+=lambda_scp*scp_loss
    else:
      pass
    
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

    loss+=total_loss.item()
    if (epoch+1)%50==0:
      loss=loss/50
      print('\r','train task {} [{}] loss:{:.3f} '.format(task.name,(total_epochs+1+epoch),loss),end=' ')
      losses.append(loss)
      loss=0.

  return model,optimizer,losses

In [ ]:
def val(model,task):
  model.eval()
  correct_cnt=0
  for imgs,labels in task.val_loader:
    imgs,labels=imgs.to(device),labels.to(device)
    outputs=model(imgs)
    _,pred_label=torch.max(outputs.data,1)
    correct_cnt+=(pred_label==labels.data).sum().item()
  return correct_cnt/task.val_dataset_size

In [ ]:
def train_process(model,optimizer,tasks,config):
  task_loss,acc={},{}
  for task_id,task in enumerate(tasks):
    print('\n')
    total_epochs=0
    task_loss[task.name]=[]
    acc[task.name]=[]
    if config.mode=='basic' or task_id==0:
      while (total_epochs<config.num_epochs):
        model,optimizer,losses=normal_train(model,optimizer,task,total_epochs,config.summary_epochs)
        task_loss[task.name]+=losses

        for subtask in range(task_id+1):
          acc[tasks[subtask].name].append(val(model,tasks[subtask]))

        total_epochs+=config.summary_epochs
        if total_epochs%config.store_epochs==0 or total_epochs>=config.num_epochs:
          save_model(model,optimizer,config.store_model_path)

    if config.mode=='ewc' and task_id>0:
      old_dataloaders=[]
      for old_task in range(task_id):
        old_dataloaders+=[tasks[old_task].val_loader]
      ewc=EWC(model,old_dataloaders,device)
      while (total_epochs<config.num_epochs):
        model,optimizer,losses=ewc_train(model,optimizer,task,total_epochs,config.summary_epochs,ewc,config.lifelong_coeff)
        task_loss[task.name]+=losses

        for subtask in range(task_id+1):
          acc[tasks[subtask].name].append(val(model,tasks[subtask]))

        total_epochs+=config.summary_epochs
        if total_epochs%config.store_epochs==0 or total_epochs>=config.num_epochs:
          save_model(model,optimizer,config.store_model_path)

    if config.mode=='mas' and task_id>0:
      old_dataloaders=[]
      mas_tasks=[]
      for old_task in range(task_id):
        old_dataloaders+=[tasks[old_task].val_loader]
        mas=MAS(model,old_dataloaders,device)
        mas_tasks+=[mas]
      while (total_epochs<config.num_epochs):
        model,optimizer,losses=mas_train(model,optimizer,task,total_epochs,config.summary_epochs,mas_tasks,config.lifelong_coeff)
        task_loss[task.name]+=losses

        for subtask in range(task_id+1):
          acc[tasks[subtask].name].append(val(model,tasks[subtask]))

        total_epochs+=config.summary_epochs
        if total_epochs%config.store_epochs==0 or total_epochs>=config.num_epochs:
          save_model(model,optimizer,config.store_model_path)

    if config.mode=='scp' and task_id>0:
      old_dataloaders=[]
      scp_tasks=[]
      for old_task in range(task_id):
        old_dataloaders+=[tasks[old_task].val_loader]
        scp=SCP(model,old_dataloaders,100,device)
        scp_tasks+=[scp]
      while (total_epochs<config.num_epochs):
        model,optimizer,losses=scp_train(model,optimizer,task,total_epochs,config.summary_epochs,scp_tasks,config.lifelong_coeff)
        task_loss[task.name]+=losses

        for subtask in range(task_id+1):
          acc[tasks[subtask].name].append(val(model,tasks[subtask]))

        total_epochs+=config.summary_epochs
        if total_epochs%config.store_epochs==0 or total_epochs>=config.num_epochs:
          save_model(model,optimizer,config.store_model_path)

  return task_loss,acc